In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

2023-01-09 16:53:37.683698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def prepereData():
    allData = pd.read_csv("data/songs_with_genre.csv", index_col=0,header=[0, 1, 2])
    generes = allData["genre_id"]
    genres_index = pd.read_csv("data/genres_no_zeroes.csv", index_col=0)
    generes.columns = range(generes.shape[1])
    allData.drop(columns=["genre_id"], inplace=True)
    data_x = allData
    data_x.columns = range(data_x.shape[1])
    curr = genres_index['genre_index']
    late = generes[0].unique().tolist()
    update_genres = {late[i]:curr[i] for i in range(len(curr))}
    #normalizing
    mu = np.mean(data_x, axis=0)
    sigma = (np.std(data_x, axis=0))
    data_x = (data_x - mu) / sigma
    #Creating the right lables
    generes.columns = range(generes.shape[1])
    generes[0]=generes[0].map(update_genres)
    targets = OneHotEncoder().fit_transform(genres_index).toarray()
    #generes[0] = generes[0] - 1
    data_y = [targets[i - 1] for i in generes[0]]
    generes[0] = data_y
    print(generes)
    test = generes.copy()
    test.rename(columns={0: "oneHotVector"}, inplace=True)
    test.shape
    split = pd.DataFrame(test["oneHotVector"].to_list(), columns=range(16))
    split.index = generes.index
    generes = split
    return data_x, generes

In [3]:
data, labels = prepereData()
#data = data.reshape(-1, 1, 518)

x_train, x_rest, y_train, y_rest = train_test_split(data, labels, train_size=0.8)
x_test, x_valid, y_test, y_valid = train_test_split(x_rest, y_rest, train_size=0.5)

                                                          0
track_id                                                   
2         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
10        [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
134       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
...                                                     ...
155315    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
155316    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
155317    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
155318    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
155319    [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

[49598 rows x 1 columns]


In [49]:
CNN = models.Sequential()
#CNN.add(layers.InputLayer(input_shape=(x_train.shape[1],)))
CNN.add(layers.Conv1D(32, 3, activation='relu',input_shape=(518,1),padding='SAME'))
CNN.add(layers.MaxPooling1D(3))
CNN.add(layers.Conv1D(64, 4, activation='relu'))
CNN.add(layers.MaxPooling1D(4))
CNN.add(layers.Flatten())
#CNN.add(layers.Conv1D(128, 5, activation='relu'))
#CNN.add(layers.Dense(256, activation='relu'))
#CNN.add(layers.Dense(128, activation='relu'))
#CNN.add(layers.Dense(6528, activation='relu'))
CNN.add(layers.Dense(16,activation='softmax'))

2023-01-09 16:50:45.439751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 16:50:45.563477: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [50]:
CNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 518, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 172, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 169, 64)           8256      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 42, 64)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 2688)              0         
                                                                 
 dense (Dense)               (None, 16)                4

In [51]:
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',mode='max',patience=10,verbose=1),tf.keras.callbacks.EarlyStopping(monitor='loss',mode='min',patience=10,verbose=1)]
CNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [52]:
training = CNN.fit(x_train,y_train,epochs=100, validation_data=(x_valid,y_valid),batch_size=389,callbacks=callback)

Epoch 1/100
 90/102 [=========================>....] - ETA: 2s - loss: 1.6292 - accuracy: 0.4767

KeyboardInterrupt: 

In [72]:
x_train.shape

(39678, 518)

In [82]:
print(training.history)

{'loss': [1.178002119064331, 1.0607845783233643, 0.9766632914543152, 0.8937786221504211, 0.807569146156311, 0.7184015512466431, 0.614281952381134, 0.5079550743103027, 0.40203070640563965, 0.3137299716472626], 'accuracy': [0.6223599910736084, 0.660466730594635, 0.6820656061172485, 0.7062855958938599, 0.7317405343055725, 0.7593124508857727, 0.792630672454834, 0.8281919360160828, 0.8626694679260254, 0.8942990899085999], 'val_loss': [1.1229056119918823, 1.0694172382354736, 1.0472040176391602, 1.0581305027008057, 1.0576930046081543, 1.0892432928085327, 1.1394011974334717, 1.2091081142425537, 1.3652393817901611, 1.43087637424469], 'val_accuracy': [0.6409274339675903, 0.6540322303771973, 0.6618951559066772, 0.657056450843811, 0.6524193286895752, 0.6538306474685669, 0.6610887050628662, 0.6514112949371338, 0.6473790407180786, 0.6411290168762207]}


In [67]:
y_train

statistics,Unnamed: 519_level_1
number,Unnamed: 519_level_2
track_id,
44204,11
16191,11
27388,14
59663,1
85101,31
...,...
34204,11
33552,20


In [ ]:
test_loss, test_acc = CNN.evaluate(x_test,y_test)